#### patient_clinical_summary

**High interview + exam value**

Business Questions Answered 

- Who is the patient?
- What conditions do they have?
- Latest labs?
- Active medications?

In [0]:
%sql
CREATE OR REPLACE TABLE medilakehealth_catalog.gold.patient_clinical_summary
USING DELTA
AS
SELECT
  p.patient_id,
  p.first_name,
  p.last_name,
  p.gender,
  p.city,

  COLLECT_SET(d.diagnosis_desc) AS conditions,

  MAX(l.result_date) AS latest_lab_date,
  MAX_BY(l.test_value, l.result_date) AS latest_lab_value,

  COLLECT_SET(m.medication_name) AS active_medications

FROM medilakehealth_catalog.silver.patients_scd p
LEFT JOIN medilakehealth_catalog.silver.diagnoses_clean d
  ON p.patient_id = d.patient_id
LEFT JOIN medilakehealth_catalog.silver.lab_results_clean l
  ON p.patient_id = l.patient_id
LEFT JOIN medilakehealth_catalog.silver.medications_clean m
  ON p.patient_id = m.patient_id
  AND m.is_active = true

WHERE p.is_current = true
GROUP BY
  p.patient_id, p.first_name, p.last_name, p.gender, p.city;


##### hospital_utilization_metrics
**Business Metrics**

- Daily admissions
- Avg length of stay
- Emergency vs Planned %

In [0]:
%sql
CREATE OR REPLACE TABLE medilakehealth_catalog.gold.hospital_utilization_metrics
USING DELTA
AS
SELECT
  hospital_id,
  encounter_date,
  COUNT(*) AS total_encounters,
  AVG(DATEDIFF(discharge_date, encounter_date)) AS avg_length_of_stay,
  SUM(CASE WHEN admission_type = 'Emergency' THEN 1 ELSE 0 END) AS emergency_cases
FROM medilakehealth_catalog.silver.encounters_clean
GROUP BY hospital_id, encounter_date;


##### disease_prevalence_daily
Classic healthcare analytics use case


In [0]:
%sql
CREATE OR REPLACE TABLE medilakehealth_catalog.gold.disease_prevalence_daily
USING DELTA
AS
SELECT
  diagnosis_date,
  diagnosis_desc,
  COUNT(DISTINCT patient_id) AS patient_count
FROM medilakehealth_catalog.silver.diagnoses_clean
GROUP BY diagnosis_date, diagnosis_desc;


##### lab_abnormal_trends
Rule
Lab value > normal range → abnormal


In [0]:
%sql
CREATE OR REPLACE TABLE medilakehealth_catalog.gold.lab_abnormal_trends
USING DELTA
AS
SELECT
  test_name,
  result_date,
  COUNT(*) AS abnormal_count
FROM medilakehealth_catalog.silver.lab_results_clean
WHERE test_value > CAST(REGEXP_REPLACE(normal_range, '[^0-9.]', '') AS DOUBLE)
GROUP BY test_name, result_date;


##### Gold Layer Performance (Small but Important)

In [0]:
%sql

OPTIMIZE medilakehealth_catalog.gold.patient_clinical_summary;



In [0]:
%sql
OPTIMIZE medilakehealth_catalog.gold.hospital_utilization_metrics;